In [24]:
import sys  
import json
import pandas as pd
import numpy as np
import lightgbm as lgb

sys.path.insert(0, '../')
import gbdt_forecast

In [25]:
params_path = '../params/params_eem2020_wind_example.json'
with open(params_path, 'r', encoding='utf-8') as file:
    params_json = json.loads(file.read())

In [26]:
params_json

{'trial_name': 'trial0',
 'trial_comment': 'test',
 'path_result': './result/eem20/',
 'path_raw_data': './data/eem20/',
 'filename_raw_data': 'eem2020-raw.csv',
 'path_preprocessed_data': './data/eem20/preprocessed/',
 'filename_preprocessed_data': 'data_final_zero_one_normalized.csv',
 'data_resolution': '60min',
 'splits': {'train': [['2000-01-01 01:00:00', '2000-10-01 00:00:00']],
  'valid': [['2000-11-01 01:00:00', '2000-12-31 00:00:00']]},
 'sites': ['SE1', 'SE2', 'SE3', 'SE4'],
 'features': ['Temperature_0',
  'RelativeHumidity_0',
  'Pressure_0',
  'CloudCover_0',
  'WindGustSpeed_0',
  'WindSpeed_0',
  'WindDirection_0'],
 'target': 'Power',
 'diff_target_with_physical': False,
 'target_smoothing_window': 1,
 'regression_params': {'type': ['quantile'],
  'alpha_range': [0.1, 1.0, 0.1],
  'quantile_postprocess': 'sorting',
  'target_min_max': [None, None]},
 'model_params': {'lightgbm': {'learning_rate': 0.1,
   'num_trees': 20,
   'boosting': 'gbdt',
   'max_leaves': 32,
   'm

In [68]:
trial_name = 'trial0'
model = 'lightgbm'

sites = range(len(params_json['sites']))
splits = range(len(params_json['splits']['valid']))
alpha_q = np.arange(params_json['regression_params']['alpha_range'][0],
                    params_json['regression_params']['alpha_range'][1],
                    params_json['regression_params']['alpha_range'][2])

gbm_site = []
for site in sites:
    gbm_split = []
    for split in splits:
        gbm_q = {}
        for alpha in alpha_q:
            file_name = '../result/eem20/'+trial_name+'/gbm_model/gbm_model_'+model+'_q_quantile{0:.2f}_split_{1}_site_{2}.txt'.format(alpha, split, site)
            if model == 'lightgbm': 
                gbm = lgb.Booster(model_file=file_name)
            elif model == 'catboost':
                gbm = cb.CatBoostRegressor().load_model(file_name)
            gbm_q['quantile{0:.2f}'.format(alpha)] = gbm
        
        gbm_split.append(gbm_q)
    gbm_site.append(gbm_split)

In [69]:
df_X = pd.read_csv('../data/eem20/preprocessed/data_final_zero_one_normalized.csv', index_col=[0,1], header=[0,1])

In [70]:
trial = gbdt_forecast.Trial(params_json)

In [71]:
df_X

SE1                                \
                                 Temperature_0 RelativeHumidity_0 Pressure_0   
ref_datetime valid_datetime                                                    
2000-01-01   2000-01-01 00:00:00     270.14926           0.981022   99697.21   
             2000-01-01 01:00:00     270.06406           0.980828   99654.18   
             2000-01-01 02:00:00     270.04492           0.980874   99614.89   
             2000-01-01 03:00:00     270.01850           0.981637   99560.07   
             2000-01-01 04:00:00     270.02536           0.983668   99513.10   
...                                        ...                ...        ...   
2000-12-31   2000-12-31 19:00:00     267.19565           0.715043  100252.07   
             2000-12-31 20:00:00     266.94232           0.711884  100453.88   
             2000-12-31 21:00:00     266.61667           0.725288  100651.61   
             2000-12-31 22:00:00     266.38275           0.722083  100846.75   
             2000-12-31 23:00:00     266.11154           0.723454  101044.13   

                                                                           \
                                 CloudCover_0 WindGustSpeed_0 WindSpeed_0   
ref_datetime valid_datetime                                                 
2000-01-01   2000-01-01 00:00:00     0.999979        9.486829    4.993632   
             2000-01-01 01:00:00     1.000000        9.282259    4.502751   
             2000-01-01 02:00:00     1.000000        8.298201    4.256284   
             2000-01-01 03:00:00     1.000000        7.815304    4.185637   
             2000-01-01 04:00:00     1.000000        7.550981    3.995219   
...                                       ...             ...         ...   
2000-12-31   2000-12-31 19:00:00     0.847345       19.014013    8.738104   
             2000-12-31 20:00:00     0.841644       19.204440    8.693635   
             2000-12-31 21:00:00     0.487979       18.930569    8.499709   
             2000-12-31 22:00:00     0.353604       18.418383    8.178779   
             2000-12-31 23:00:00     0.304413       17.753016    7.627296   

                                                           SE2  \
                                 WindDirection_0 Temperature_0   
ref_datetime valid_datetime                                      
2000-01-01   2000-01-01 00:00:00      278.587777     266.29095   
             2000-01-01 01:00:00      285.627465     266.81268   
             2000-01-01 02:00:00      293.956715     267.31152   
             2000-01-01 03:00:00      298.041503     267.78010   
             2000-01-01 04:00:00      303.615424     268.19745   
...                                          ...           ...   
2000-12-31   2000-12-31 19:00:00       93.895614     267.66162   
             2000-12-31 20:00:00       96.327828     267.42080   
             2000-12-31 21:00:00       96.315182     267.11243   
             2000-12-31 22:00:00       96.320073     266.77400   
             2000-12-31 23:00:00       99.055959     266.46927   

                                                                ...  \
                                 RelativeHumidity_0 Pressure_0  ...   
ref_datetime valid_datetime                                     ...   
2000-01-01   2000-01-01 00:00:00           0.990853   99131.40  ...   
             2000-01-01 01:00:00           0.989967   99030.55  ...   
             2000-01-01 02:00:00           0.990838   98937.67  ...   
             2000-01-01 03:00:00           0.990665   98838.88  ...   
             2000-01-01 04:00:00           0.989776   98761.58  ...   
...                                             ...        ...  ...   
2000-12-31   2000-12-31 19:00:00           0.677650  100806.57  ...   
             2000-12-31 20:00:00           0.639092  101038.80  ...   
             2000-12-31 21:00:00           0.618090  101254.39  ...   
             2000-12-31 22:00:00           0.623507  101451.29  ..

In [77]:
df_X

SE1                                \
                                 Temperature_0 RelativeHumidity_0 Pressure_0   
ref_datetime valid_datetime                                                    
2000-01-01   2000-01-01 00:00:00     270.14926           0.981022   99697.21   
             2000-01-01 01:00:00     270.06406           0.980828   99654.18   
             2000-01-01 02:00:00     270.04492           0.980874   99614.89   
             2000-01-01 03:00:00     270.01850           0.981637   99560.07   
             2000-01-01 04:00:00     270.02536           0.983668   99513.10   
...                                        ...                ...        ...   
2000-12-31   2000-12-31 19:00:00     267.19565           0.715043  100252.07   
             2000-12-31 20:00:00     266.94232           0.711884  100453.88   
             2000-12-31 21:00:00     266.61667           0.725288  100651.61   
             2000-12-31 22:00:00     266.38275           0.722083  100846.75   
             2000-12-31 23:00:00     266.11154           0.723454  101044.13   

                                                                           \
                                 CloudCover_0 WindGustSpeed_0 WindSpeed_0   
ref_datetime valid_datetime                                                 
2000-01-01   2000-01-01 00:00:00     0.999979        9.486829    4.993632   
             2000-01-01 01:00:00     1.000000        9.282259    4.502751   
             2000-01-01 02:00:00     1.000000        8.298201    4.256284   
             2000-01-01 03:00:00     1.000000        7.815304    4.185637   
             2000-01-01 04:00:00     1.000000        7.550981    3.995219   
...                                       ...             ...         ...   
2000-12-31   2000-12-31 19:00:00     0.847345       19.014013    8.738104   
             2000-12-31 20:00:00     0.841644       19.204440    8.693635   
             2000-12-31 21:00:00     0.487979       18.930569    8.499709   
             2000-12-31 22:00:00     0.353604       18.418383    8.178779   
             2000-12-31 23:00:00     0.304413       17.753016    7.627296   

                                                           SE2  \
                                 WindDirection_0 Temperature_0   
ref_datetime valid_datetime                                      
2000-01-01   2000-01-01 00:00:00      278.587777     266.29095   
             2000-01-01 01:00:00      285.627465     266.81268   
             2000-01-01 02:00:00      293.956715     267.31152   
             2000-01-01 03:00:00      298.041503     267.78010   
             2000-01-01 04:00:00      303.615424     268.19745   
...                                          ...           ...   
2000-12-31   2000-12-31 19:00:00       93.895614     267.66162   
             2000-12-31 20:00:00       96.327828     267.42080   
             2000-12-31 21:00:00       96.315182     267.11243   
             2000-12-31 22:00:00       96.320073     266.77400   
             2000-12-31 23:00:00       99.055959     266.46927   

                                                                ...  \
                                 RelativeHumidity_0 Pressure_0  ...   
ref_datetime valid_datetime                                     ...   
2000-01-01   2000-01-01 00:00:00           0.990853   99131.40  ...   
             2000-01-01 01:00:00           0.989967   99030.55  ...   
             2000-01-01 02:00:00           0.990838   98937.67  ...   
             2000-01-01 03:00:00           0.990665   98838.88  ...   
             2000-01-01 04:00:00           0.989776   98761.58  ...   
...                                             ...        ...  ...   
2000-12-31   2000-12-31 19:00:00           0.677650  100806.57  ...   
             2000-12-31 20:00:00           0.639092  101038.80  ...   
             2000-12-31 21:00:00           0.618090  101254.39  ...   
             2000-12-31 22:00:00           0.623507  101451.29  ..

In [79]:
dfs_y_pred = trial.predict_model_split_site([[df_X['SE1'], df_X['SE2'], df_X['SE3'], df_X['SE4']]], {model: gbm_site})


 25%|██▌       | 1/4 [00:00<00:00,  7.55it/s]

Predicting...


In [87]:
dfs_y_pred['lightgbm'][0][0]

quantile10  quantile20  quantile30  \
ref_datetime valid_datetime                                            
2000-01-01   2000-01-01 00:00:00    0.226095    0.299862    0.398635   
             2000-01-01 01:00:00    0.224595    0.299862    0.400803   
             2000-01-01 02:00:00    0.183567    0.233735    0.308788   
             2000-01-01 03:00:00    0.183567    0.233735    0.308788   
             2000-01-01 04:00:00    0.153018    0.233735    0.308788   
...                                      ...         ...         ...   
2000-12-31   2000-12-31 19:00:00    0.276491    0.419746    0.521715   
             2000-12-31 20:00:00    0.276491    0.419746    0.521715   
             2000-12-31 21:00:00    0.276491    0.419746    0.521715   
             2000-12-31 22:00:00    0.276491    0.419746    0.521715   
             2000-12-31 23:00:00    0.276491    0.419746    0.497583   

                                  quantile40  quantile50  quantile60  \
ref_datetime valid_datetime                                            
2000-01-01   2000-01-01 00:00:00    0.444501    0.455626    0.458126   
             2000-01-01 01:00:00    0.434786    0.440294    0.444928   
             2000-01-01 02:00:00    0.324575    0.331211    0.333070   
             2000-01-01 03:00:00    0.324575    0.331211    0.333070   
             2000-01-01 04:00:00    0.317847    0.321932    0.331108   
...                                      ...         ...         ...   
2000-12-31   2000-12-31 19:00:00    0.613932    0.680765    0.730058   
             2000-12-31 20:00:00    0.613932    0.680765    0.730058   
             2000-12-31 21:00:00    0.613932    0.680765    0.736653   
             2000-12-31 22:00:00    0.613932    0.680765    0.736653   
             2000-12-31 23:00:00    0.613932    0.666888    0.723236   

                                  quantile70  quantile80  quantile90  
ref_datetime valid_datetime                                           
2000-01-01   2000-01-01 00:00:00    0.502574    0.571145    0.706504  
             2000-01-01 01:00:00    0.532432    0.584472    0.640881  
             2000-01-01 02:00:00    0.348150    0.431088    0.556552  
             2000-01-01 03:00:00    0.348150    0.431088    0.556552  
             2000-01-01 04:00:00    0.353191    0.419178    0.507733  
...                                      ...         ...         ...  
2000-12-31   2000-12-31 19:00:00    0.733286    0.736653    0.789329  
             2000-12-31 20:00:00    0.733286    0.736653    0.789329  
             2000-12-31 21:00:00    0.744846    0.746080    0.774568  
             2000-12-31 22:00:00    0.742476    0.744846    0.795027  
             2000-12-31 23:00:00    0.736653    0.747800    0.803502  

[8736 rows x 9 columns]